# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f173c3e1ca0>
├── 'a' --> tensor([[ 0.6505, -1.0021,  0.1281],
│                   [ 0.4797,  1.7353,  0.2242]])
└── 'x' --> <FastTreeValue 0x7f173c3e1c70>
    └── 'c' --> tensor([[-2.5782,  0.2430,  0.5118, -0.6815],
                        [ 1.6398,  0.4957, -1.3584, -0.1097],
                        [-0.6705, -0.3416, -0.0395, -0.2628]])

In [4]:
t.a

tensor([[ 0.6505, -1.0021,  0.1281],
        [ 0.4797,  1.7353,  0.2242]])

In [5]:
%timeit t.a

57.5 ns ± 0.0825 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f173c3e1ca0>
├── 'a' --> tensor([[-1.4077, -1.1430, -0.8441],
│                   [ 0.7987,  0.3273,  0.4108]])
└── 'x' --> <FastTreeValue 0x7f173c3e1c70>
    └── 'c' --> tensor([[-2.5782,  0.2430,  0.5118, -0.6815],
                        [ 1.6398,  0.4957, -1.3584, -0.1097],
                        [-0.6705, -0.3416, -0.0395, -0.2628]])

In [7]:
%timeit t.a = new_value

60.9 ns ± 0.101 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.6505, -1.0021,  0.1281],
               [ 0.4797,  1.7353,  0.2242]]),
    x: Batch(
           c: tensor([[-2.5782,  0.2430,  0.5118, -0.6815],
                      [ 1.6398,  0.4957, -1.3584, -0.1097],
                      [-0.6705, -0.3416, -0.0395, -0.2628]]),
       ),
)

In [10]:
b.a

tensor([[ 0.6505, -1.0021,  0.1281],
        [ 0.4797,  1.7353,  0.2242]])

In [11]:
%timeit b.a

53.5 ns ± 0.0523 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.2027,  0.1057,  0.1849],
               [ 0.2676,  0.7527, -1.1432]]),
    x: Batch(
           c: tensor([[-2.5782,  0.2430,  0.5118, -0.6815],
                      [ 1.6398,  0.4957, -1.3584, -0.1097],
                      [-0.6705, -0.3416, -0.0395, -0.2628]]),
       ),
)

In [13]:
%timeit b.a = new_value

490 ns ± 0.359 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

834 ns ± 7.43 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 22.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 494 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 275 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f173c3daa30>
├── 'a' --> tensor([[[ 0.6505, -1.0021,  0.1281],
│                    [ 0.4797,  1.7353,  0.2242]],
│           
│                   [[ 0.6505, -1.0021,  0.1281],
│                    [ 0.4797,  1.7353,  0.2242]],
│           
│                   [[ 0.6505, -1.0021,  0.1281],
│                    [ 0.4797,  1.7353,  0.2242]],
│           
│                   [[ 0.6505, -1.0021,  0.1281],
│                    [ 0.4797,  1.7353,  0.2242]],
│           
│                   [[ 0.6505, -1.0021,  0.1281],
│                    [ 0.4797,  1.7353,  0.2242]],
│           
│                   [[ 0.6505, -1.0021,  0.1281],
│                    [ 0.4797,  1.7353,  0.2242]],
│           
│                   [[ 0.6505, -1.0021,  0.1281],
│                    [ 0.4797,  1.7353,  0.2242]],
│           
│                   [[ 0.6505, -1.0021,  0.1281],
│                    [ 0.4797,  1.7353,  0.2242]]])
└── 'x' --> <FastTreeValue 0x7f173c3da910>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.6 µs ± 74.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f173c3a6400>
├── 'a' --> tensor([[ 0.6505, -1.0021,  0.1281],
│                   [ 0.4797,  1.7353,  0.2242],
│                   [ 0.6505, -1.0021,  0.1281],
│                   [ 0.4797,  1.7353,  0.2242],
│                   [ 0.6505, -1.0021,  0.1281],
│                   [ 0.4797,  1.7353,  0.2242],
│                   [ 0.6505, -1.0021,  0.1281],
│                   [ 0.4797,  1.7353,  0.2242],
│                   [ 0.6505, -1.0021,  0.1281],
│                   [ 0.4797,  1.7353,  0.2242],
│                   [ 0.6505, -1.0021,  0.1281],
│                   [ 0.4797,  1.7353,  0.2242],
│                   [ 0.6505, -1.0021,  0.1281],
│                   [ 0.4797,  1.7353,  0.2242],
│                   [ 0.6505, -1.0021,  0.1281],
│                   [ 0.4797,  1.7353,  0.2242]])
└── 'x' --> <FastTreeValue 0x7f16856dc820>
    └── 'c' --> tensor([[-2.5782,  0.2430,  0.5118, -0.6815],
                        [ 1.6398,  0.4957, -1.3584, -0.1097],
                 

In [23]:
%timeit t_cat(trees)

30.8 µs ± 57.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

60 µs ± 240 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-2.5782,  0.2430,  0.5118, -0.6815],
                       [ 1.6398,  0.4957, -1.3584, -0.1097],
                       [-0.6705, -0.3416, -0.0395, -0.2628]],
              
                      [[-2.5782,  0.2430,  0.5118, -0.6815],
                       [ 1.6398,  0.4957, -1.3584, -0.1097],
                       [-0.6705, -0.3416, -0.0395, -0.2628]],
              
                      [[-2.5782,  0.2430,  0.5118, -0.6815],
                       [ 1.6398,  0.4957, -1.3584, -0.1097],
                       [-0.6705, -0.3416, -0.0395, -0.2628]],
              
                      [[-2.5782,  0.2430,  0.5118, -0.6815],
                       [ 1.6398,  0.4957, -1.3584, -0.1097],
                       [-0.6705, -0.3416, -0.0395, -0.2628]],
              
                      [[-2.5782,  0.2430,  0.5118, -0.6815],
                       [ 1.6398,  0.4957, -1.3584, -0.1097],
                       [-0.6705, -0.3416, -0.0395, -0.2628]],

In [26]:
%timeit Batch.stack(batches)

77.6 µs ± 147 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-2.5782,  0.2430,  0.5118, -0.6815],
                      [ 1.6398,  0.4957, -1.3584, -0.1097],
                      [-0.6705, -0.3416, -0.0395, -0.2628],
                      [-2.5782,  0.2430,  0.5118, -0.6815],
                      [ 1.6398,  0.4957, -1.3584, -0.1097],
                      [-0.6705, -0.3416, -0.0395, -0.2628],
                      [-2.5782,  0.2430,  0.5118, -0.6815],
                      [ 1.6398,  0.4957, -1.3584, -0.1097],
                      [-0.6705, -0.3416, -0.0395, -0.2628],
                      [-2.5782,  0.2430,  0.5118, -0.6815],
                      [ 1.6398,  0.4957, -1.3584, -0.1097],
                      [-0.6705, -0.3416, -0.0395, -0.2628],
                      [-2.5782,  0.2430,  0.5118, -0.6815],
                      [ 1.6398,  0.4957, -1.3584, -0.1097],
                      [-0.6705, -0.3416, -0.0395, -0.2628],
                      [-2.5782,  0.2430,  0.5118, -0.6815],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 516 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

308 µs ± 680 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
